In [34]:
import wikipedia 
import transformers
from pathlib import Path 
from transformers import AutoModelForCausalLM, AutoTokenizer
transformers.utils.logging.set_verbosity(transformers.logging.CRITICAL)



# My personal cache directory
cache_dir = Path('/n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache')
data_cache= Path("/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia")

if not cache_dir.exists():
    cache_dir = None 
if not data_cache.exists():
    data_cache = None 
    

import os
os.environ["HF_HOME"] = str(cache_dir)


os.environ["TRANSFORMERS_CACHE"] = str(cache_dir)
os.environ["HF_DATASETS_CACHE"] = str(cache_dir)
os.environ["HF_HUB_CACHE"] = str(cache_dir)

In [35]:
import torch 
from transformers.utils import logging
logging.set_verbosity_debug()

model_id = 'HuggingFaceH4/zephyr-7b-beta'

#model_id = "mistralai/Mistral-7B-Instruct-v0.3"
print(f"model_id- {model_id}")
print(f"cache_dir - {cache_dir}")   
device = 'cuda:0'
dtype= torch.float32
model = AutoModelForCausalLM.from_pretrained(model_id,
                 
                                             torch_dtype=dtype, cache_dir=cache_dir,)
model = model.to(device)
model.requires_grad_(False)
tokenizer = AutoTokenizer.from_pretrained(model_id, 
                                          use_fast=False, cache_dir=cache_dir)

loading configuration file config.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_cache": true,
  "vocab_size": 32000
}



model_id- HuggingFaceH4/zephyr-7b-beta
cache_dir - /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache


Detected torch version: 2.5.1
Detected torch version: 2.5.1
loading weights file model.safetensors from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/model.safetensors.index.json
Instantiating MistralForCausalLM model under default dtype torch.float32.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 2
}



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at HuggingFaceH4/zephyr-7b-beta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

loading file tokenizer.model from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/tokenizer.model
loading file added_tokens.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b

In [36]:
from importlib import reload 

from wiki_rag import wikipedia
reload(wikipedia)
from wiki_rag import rag

import os

from pathlib import Path
data_cache= Path("/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/text")





for i, article in enumerate(wikipedia.parse_wikiextractor_output(data_cache)):
    print(article['title'])
    print(article.keys())
    
    text = article['text']
    if len(text) < 100:
        continue
    
    # remove any leading or trailing whitespace
    text = text.strip()
    print(len(text))
    print(text[:1000])  # preview first 300 characters
    if i >10:
        break  # remove this to keep going


AccessibleComputing
dict_keys(['id', 'title', 'url', 'text'])
Anarchism
dict_keys(['id', 'title', 'url', 'text'])
45652
Anarchism

Anarchism is a political philosophy and movement that seeks to abolish all institutions that perpetuate authority, coercion, or hierarchy, primarily targeting the state and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. A historically left-wing movement, anarchism is usually described as the libertarian wing of the socialist movement (libertarian socialism).
Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a significant role in workers' struggles for emancipation. Various anarchist schools of thought formed during this period. Anarchists have taken part in several revolu

In [59]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
from typing import List
import numpy as np
from pathlib import Path 
import json
import os
from wiki_rag import wikipedia
from itertools import islice
from typing import Iterator


# === Helper to batch an iterator ===
def batched(iterable: Iterator, batch_size: int):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

embeddings = rag.ModelEmbeddings(model, tokenizer, device)
vectorstore = None

wiki_generator = wikipedia.parse_wikiextractor_output(data_cache)
counts = 0
batch_size = 4

docs = []
    
for i, d in enumerate(wiki_generator):
    #print(f"Processing article {counts}: {d['title']}")
    if i > int(2000):
        break

    title = d['title']
    url = d['url']
    text = d['text']
    id_ = d.get('id')
    if len(text) < 100:
        continue
    counts +=1
    
    if counts % 25 == 0:
        print(f"Processed {counts} articles so far...")
    
    text = text.strip()
    # abstract is first 3 par
    abstract = "\n".join(text.split("\n")[:3])
    doc = Document(page_content=abstract, metadata={"title": title})
    
    if vectorstore is None:
        vectorstore = FAISS.from_documents([doc], embeddings)
    else:
        vectorstore.add_documents([doc])

index_path = "wiki_faiss_index"
if vectorstore:
    vectorstore.save_local(index_path)
    print(f"✅ FAISS index saved to {index_path}")
else:
    print("⚠️ No documents were indexed.")


Processed 25 articles so far...
Processed 50 articles so far...
Processed 75 articles so far...
Processed 100 articles so far...
Processed 125 articles so far...
Processed 150 articles so far...
Processed 175 articles so far...
Processed 200 articles so far...
Processed 225 articles so far...
Processed 250 articles so far...
Processed 275 articles so far...
Processed 300 articles so far...
Processed 325 articles so far...
Processed 350 articles so far...
Processed 375 articles so far...
Processed 400 articles so far...
Processed 425 articles so far...
Processed 450 articles so far...
Processed 475 articles so far...
Processed 500 articles so far...
Processed 525 articles so far...
Processed 550 articles so far...
Processed 575 articles so far...
Processed 600 articles so far...
Processed 625 articles so far...
Processed 650 articles so far...
Processed 675 articles so far...
Processed 700 articles so far...
Processed 725 articles so far...
Processed 750 articles so far...
Processed 775

In [53]:
vectorstore.search("Who is the author of the article on quantum computing?", k=5, search_type="similarity")

#num_elements = 
vectorstore.index.ntotal


2

In [ ]:

wiki_generator = wikipedia.wikipedia_abstract_generator(data_cache)

for _ in range(3):
    print(next(wiki_generator))

AttributeError: 'str' object has no attribute 'get'

In [ ]:
import datasets
from tqdm.notebook import tqdm
import numpy as np
import torch
# from transformers import AdamW
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss,MSELoss, NLLLoss, KLDivLoss
import json
import random
import matplotlib.pyplot as plt